In [1]:
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np

In [2]:
#создаем список URL
URL = 'https://www.kommersant.ru/doc' # базовая ссылка
FIRST_PAGE = 4866471 # номер  страницы с первой новостью
LAST_PAGE = 4866521 # номер страницы с последней новостью

urls_list = [] # создаем пустой список
for page in range(FIRST_PAGE, LAST_PAGE): # перебираем страницы 
    page_url = '{}/{}'.format(URL, page)
    urls_list.append(page_url) # записываем адреса в список
    
print('Длина полученного списка веб-страниц: {}'.format(len(urls_list)))
print('Пример записанных в список страниц: {}'.format(urls_list[0]))

Длина полученного списка веб-страниц: 50
Пример записанных в список страниц: https://www.kommersant.ru/doc/4866471


In [3]:
page = requests.get(urls_list[0]) # попробуем спарсить одну страницу
print(page.status_code) # если 200, то успешно подключились к странице

200


In [4]:
soup = BeautifulSoup(page.text, "html.parser") # посмотрим как выглядит спарсенная страница
soup


<!DOCTYPE html>

<html class="no-js" lang="ru" prefix="og: http://ogp.me/ns# article: http://ogp.me/ns/article#">
<head>
<title>Президент Германии оценил «масштабы одного преступления» – Мир – Коммерсантъ</title>
<meta content="IE=edge, chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<script> if (document.cookie.indexOf('IsNonMobileViewport') >= 0){document.write('<meta name="viewport" content="width=1200">')}</script>
<meta content="telephone=no" name="format-detection">
<meta content='Президент Германии оценил «масштабы одного преступления» - Открытие выставки о советских военнопленных не обошлось без скандала - "Коммерсантъ" - Коммерсантъ: последние новости России и мира' name="title"/>
<meta content="Немцы обязаны сохранять память об «убийственном варварстве» в ХХ веке — развязанной нацистами Второй мировой войне. Таким оказался ключевой посыл выступления президента ФРГ Франка-Вальтера Штайнма

Всегда нужно изучить html-состав страницы, чтобы правильно образаться к тем или иным полям. Например, в Google Chrome, можно выделить элемент и нажать 'Inspect' для получения соответствующего html кода

In [7]:
%%time

titles_list = [] # создаем список заголовков
abstracts_list = [] # создаем список абстрактов
texts_list = [] # создаем список текстов статей
authors_list = [] # создаем список авторов статей
dates_list = [] # создаем список дат 

for i, url in enumerate(urls_list):
    page = requests.get(url) # отправляем запрос
    if page.status_code == 200: # если ссылка рабочая, код = 200
        soup = BeautifulSoup(page.text, 'html.parser') # парсим страницу новостей
    else:
        continue
    title = soup.find_all('h1', class_='article_name') # ищем название статьи
    if title:
        titles_list.append(title[0].text)
    else:
        titles_list.append(np.nan)

    text = soup.find_all('p', class_='b-article__text') # ищем текст статьи
    if text:
        abstracts_list.append(text[0].text)
        texts_list.append(''.join([par.text for par in text[1:]]))
    else:
        abstracts_list.append(np.nan)
        texts_list.append(np.nan)
    
    authors = soup.find_all('p', class_='b-article__text document_authors') # ищем авторов статьи
    if authors:
        authors_list.append(authors[0].text)
    else:
        authors_list.append(np.nan)
    
    date = soup.find_all('meta', itemprop='datePublished') # ищем дату публикации статьи
    if date:
        dates_list.append(date[0]['content'])
    else:
        dates_list.append(np.nan)
        
    if (i+1)%5 ==0:
        print('Кол-во спарсенных новостей: {}'.format(i+1)) # печатаем через каждые 5 новостей
    time.sleep(2.0) # пауза для обхода блокировки 

Кол-во спарсенных новостей: 5
Кол-во спарсенных новостей: 10
Кол-во спарсенных новостей: 15
Кол-во спарсенных новостей: 20
Кол-во спарсенных новостей: 25
Кол-во спарсенных новостей: 30
Кол-во спарсенных новостей: 35
Кол-во спарсенных новостей: 40
Кол-во спарсенных новостей: 45
Кол-во спарсенных новостей: 50
CPU times: user 2.55 s, sys: 97.8 ms, total: 2.65 s
Wall time: 1min 57s


In [8]:
parsed_df = pd.DataFrame({'date': dates_list, 
              'author': authors_list, 
              'title': titles_list, 
              'abstract': abstracts_list, 
              'text': texts_list}) # сформируем датафрейм

parsed_df.head() # посмотрим на спарсенный датафрейм

,date,author,title,abstract,text
0,2021-06-19T10:48:20+03:00,Галина Дудина,Президент Германии оценил «масштабы одного пре...,Немцы обязаны сохранять память об «убийственно...,«22 июня 1941 года Борис Попов встретил молоды...
1,2021-06-17T16:31:24+03:00,NaN,BMW начала тесты водородного кроссовера,Компания BMW начала тесты водородного кроссове...,Модель на топливных элементах получила установ...
2,2021-06-18T22:38:53+03:00,NaN,NaN,Правительство Российской Федерации распорядило...,Новая модель рынка тепла заработает в Перми с ...
3,2021-06-19T12:01:05+03:00,Евгений Жирнов,«Лишить Турцию фактического контроля»,"85 лет назад, 22 июня 1936 года, в Монтре откр...",«Чтобы предписать туркам наши требования»Важне...
4,2021-06-18T22:51:25+03:00,Кирилл Антонов,Татарстан выбирает режим вакцинации,В Татарстане вслед за другими регионами могут ...,О возможном введении в Татарстане обязательной...
